# Contexto

Opendata AIG Brazil

A base de dados de ocorrências aeronáuticas é gerenciada pelo Centro de Investigação e Prevenção de Acidentes Aeronáuticos (CENIPA). Constam nesta base de dados as ocorrências aeronáuticas notificadas ao CENIPA nos últimos 10 anos e que ocorreram em solo brasileiro.

Dentre as informações disponíveis estão os dados sobre as aeronaves envolvidas, fatalidades, local, data, horário dos eventos e informações taxonômicas típicas das investigações de acidentes (AIG). São resguardadas a privacidade de pessoas físicas/jurídicas envolvidas conforme previsto pela Lei de Acesso à Informação (Lei n° 12.527, de 18 de novembro de 2011).

Essa base de dados é composta por informações preliminares provenientes do formulário CENIPA-05 (Ficha de Notificação de Ocorrências Aeronáuticas) e consolidada a partir dos relatórios de investigações publicados. Outra forma de visualização desses dados é pelo Painel SIPAER disponível na página do CENIPA.

Dados proveniente dos Programas de Gerenciamento de Prevenção administrados pelo CENIPA (Emissão de Raio Laser e Risco Baloeiro) não constam nesta base. Esses programas possuem formulários próprios para coleta de dados com foco exclusivo para gerenciamento do risco, enquanto os dados coletados pelo formulário CENIPA-05 têm foco principal na investigação de acidentes (AIG).
Tabelas (arquivos)

    OCORRÊNCIA.csv - Informações sobre as ocorrências.
    OCORRÊNCIA_TIPO.csv - Informações sobre o tipo de ocorrência.
    AERONAVE.csv - Informações sobre as aeronaves envolvidas nas ocorrências.
    FATOR_CONTRIBUINTE.csv - Informações sobre os fatores contribuinte das ocorrências que tiveram investigações finalizadas.
    RECOMENDAÇÃO.csv - Informações sobre as recomendações de segurança geradas nas ocorrências.
    Fonte: Sistema DÉDALO.


## Tabela Relacionamento

<img src="http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/modelo_dados.png" width="700 px">

# 0.0. IMPORTS

In [186]:
import pandas as pd
import numpy as np
import math

from IPython.display import display
pd.options.display.max_columns = None

## 0.1. Helper Functions

## 0.2. Loading data

In [7]:
ocorrencia = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv', sep=';')
tipo_ocorrencia = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia_tipo.csv', sep=';')
aeronave = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/aeronave.csv', sep=';')
fator_contribuinte = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/fator_contribuinte.csv', sep=';')
rec_seguranca = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/recomendacao.csv', sep=';')


In [19]:
#merge
aux1 = pd.merge( ocorrencia, tipo_ocorrencia, how='left', on='codigo_ocorrencia1' )
aux2 = pd.merge( aux1, aeronave, how='left', on='codigo_ocorrencia2' )
aux3 = pd.merge( aux2, fator_contribuinte, how='left', on='codigo_ocorrencia3' )
df_raw = pd.merge( aux3, rec_seguranca, how='left', on='codigo_ocorrencia4' )



In [27]:
df_raw.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17646 entries, 0 to 17645
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   codigo_ocorrencia                17646 non-null  int64  
 1   codigo_ocorrencia1               17646 non-null  int64  
 2   codigo_ocorrencia2               17646 non-null  int64  
 3   codigo_ocorrencia3               17646 non-null  int64  
 4   codigo_ocorrencia4               17646 non-null  int64  
 5   ocorrencia_classificacao         17646 non-null  object 
 6   ocorrencia_latitude              16065 non-null  object 
 7   ocorrencia_longitude             16065 non-null  object 
 8   ocorrencia_cidade                17646 non-null  object 
 9   ocorrencia_uf                    17646 non-null  object 
 10  ocorrencia_pais                  17646 non-null  object 
 11  ocorrencia_aerodromo             17646 non-null  object 
 12  ocorrencia_dia    

<ipython-input-27-7128da69681d>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_raw.info(null_counts=True)


In [22]:
#df_raw.to_csv('./data/ocorrencias_aeronauticas_raw.csv', index=False)

In [23]:
df_raw = pd.read_csv('./data/ocorrencias_aeronauticas_raw.csv')

In [24]:
with pd.option_context('display.max_rows', 10):
    print(df_raw)

       codigo_ocorrencia  codigo_ocorrencia1  codigo_ocorrencia2  \
0                  40211               40211               40211   
1                  40349               40349               40349   
2                  40351               40351               40351   
3                  39527               39527               39527   
4                  39527               39527               39527   
...                  ...                 ...                 ...   
17641              80245               80245               80245   
17642              80247               80247               80247   
17643              80251               80251               80251   
17644              80257               80257               80257   
17645              80259               80259               80259   

       codigo_ocorrencia3  codigo_ocorrencia4 ocorrencia_classificacao  \
0                   40211               40211                INCIDENTE   
1                   40349          

# 1.0.  DESCRICAO DOS DADOS

In [88]:
df1 = df_raw.copy()

## 1.1. Rename Columns

## 1.2. Data Dimensions

In [89]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

Number of Rows: 17646
Number of Cols: 59


In [90]:
# check raw data main infos
df1.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17646 entries, 0 to 17645
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   codigo_ocorrencia                17646 non-null  int64  
 1   codigo_ocorrencia1               17646 non-null  int64  
 2   codigo_ocorrencia2               17646 non-null  int64  
 3   codigo_ocorrencia3               17646 non-null  int64  
 4   codigo_ocorrencia4               17646 non-null  int64  
 5   ocorrencia_classificacao         17646 non-null  object 
 6   ocorrencia_latitude              16065 non-null  object 
 7   ocorrencia_longitude             16065 non-null  object 
 8   ocorrencia_cidade                17646 non-null  object 
 9   ocorrencia_uf                    17646 non-null  object 
 10  ocorrencia_pais                  17646 non-null  object 
 11  ocorrencia_aerodromo             17646 non-null  object 
 12  ocorrencia_dia    

<ipython-input-90-6769e3449d77>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df1.info(null_counts=True)


In [ ]:
##Feature engineering
## Só sim e não
#divulgacao_relatorio_publicado -só tem sim e não  
#ocorrencia_saida_pista

## 1.3. Data Types

### Alterando os tipos dos dados para facilitar a manipulação de dados e reduzir o uso de recursos computacionais

In [79]:
df1.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia1                   int64
codigo_ocorrencia2                   int64
codigo_ocorrencia3                   int64
codigo_ocorrencia4                   int64
ocorrencia_classificacao            object
ocorrencia_latitude                 object
ocorrencia_longitude                object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_pais                     object
ocorrencia_aerodromo                object
ocorrencia_dia                      object
ocorrencia_hora                     object
investigacao_aeronave_liberada      object
investigacao_status                 object
divulgacao_relatorio_numero         object
divulgacao_relatorio_publicado      object
divulgacao_dia_publicacao           object
total_recomendacoes                  int64
total_aeronaves_envolvidas           int64
ocorrencia_saida_pista              object
ocorrencia_tipo                     object
ocorrencia_

In [83]:
#ocorrencia_dia - mudar para datetime 
df1.ocorrencia_dia = pd.to_datetime(df1['ocorrencia_dia'], format='%d/%m/%Y')
df1.ocorrencia_dia = pd.to_datetime(df1['ocorrencia_dia'], format='%Y-%m-%d')


# recomendacao_dia_encaminhamento - 0 se dia for data com má formato 
df1['recomendacao_dia_encaminhamento'] = df1['recomendacao_dia_encaminhamento'].apply( lambda x: 0 if ((x == '0002-11-29') or( x =='0002-11-24')or (x == '0002-11-17') or (x == '0002-11-27') or (x == '0002-11-26')) else x )
df1['recomendacao_dia_encaminhamento'] =   pd.to_datetime(df1['recomendacao_dia_encaminhamento'], format= '%Y-%m-%d')


#recomendacao_dia_feedback
df1['recomendacao_dia_feedback'] = df1['recomendacao_dia_feedback'].apply(lambda x: 0 if ((x == '0000-00-00') or (x == '0002-11-29') or (x == '0002-11-21') or (x == '0002-11-23')) else x)
df1['recomendacao_dia_feedback'] =   pd.to_datetime(df1['recomendacao_dia_feedback'], format= '%Y-%m-%d')

#recomendacao_dia_assinatura
df1['recomendacao_dia_assinatura'] =   pd.to_datetime(df1['recomendacao_dia_assinatura'], format= '%Y-%m-%d')

In [94]:
# convertendo os dados numéricos
for c in df1.select_dtypes( include=['int64'] ):
    df1[c] = df1[c].astype('int32')
    
# 36  aeronave_ano_fabricacao          17414 non-null  float64
df1['aeronave_ano_fabricacao'] = df1['aeronave_fatalidades_total'].astype('int32')

In [98]:
# 35  aeronave_assentos                17646 non-null  int32  
# aeronave_assentos - 0 para na - lidarei com os na mais tarde
df1['aeronave_assentos'].fillna(0, inplace=True )
df1.aeronave_assentos =df1['aeronave_assentos'].astype('int32')

In [100]:
# convertendo os dados categóricos
for c in df1.select_dtypes( include=['object'] ):
    df1[c] = df1[c].astype('category')

In [104]:
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17646 entries, 0 to 17645
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   codigo_ocorrencia                17646 non-null  int32   
 1   codigo_ocorrencia1               17646 non-null  int32   
 2   codigo_ocorrencia2               17646 non-null  int32   
 3   codigo_ocorrencia3               17646 non-null  int32   
 4   codigo_ocorrencia4               17646 non-null  int32   
 5   ocorrencia_classificacao         17646 non-null  category
 6   ocorrencia_latitude              16065 non-null  category
 7   ocorrencia_longitude             16065 non-null  category
 8   ocorrencia_cidade                17646 non-null  category
 9   ocorrencia_uf                    17646 non-null  category
 10  ocorrencia_pais                  17646 non-null  category
 11  ocorrencia_aerodromo             17646 non-null  category
 12  ocor

## 1.4. Check NA

In [36]:
df1.sample()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
17564,80162,80162,80162,80162,80162,INCIDENTE,-4.883167,-65.355806,COARI,AM,BRASIL,****,02/07/2021,14:34:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO,COM CARGAS EXTERNAS,COM CARGAS EXTERNAS,EXTL,PROTD,***,***,***,***,***,***,***,0,0,NaN,NaN,BRASIL,BRASIL,***,ESPECIALIZADA,PORTO URUCU,FORA DE AERODROMO,CRUZEIRO,ESPECIALIZADA,NENHUM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df1.isna().sum()

codigo_ocorrencia                      0
codigo_ocorrencia1                     0
codigo_ocorrencia2                     0
codigo_ocorrencia3                     0
codigo_ocorrencia4                     0
ocorrencia_classificacao               0
ocorrencia_latitude                 1581
ocorrencia_longitude                1581
ocorrencia_cidade                      0
ocorrencia_uf                          0
ocorrencia_pais                        0
ocorrencia_aerodromo                   0
ocorrencia_dia                         0
ocorrencia_hora                        1
investigacao_aeronave_liberada      1952
investigacao_status                  354
divulgacao_relatorio_numero          832
divulgacao_relatorio_publicado         0
divulgacao_dia_publicacao           4740
total_recomendacoes                    0
total_aeronaves_envolvidas             0
ocorrencia_saida_pista                 0
ocorrencia_tipo                        0
ocorrencia_tipo_categoria              0
taxonomia_tipo_i

## 1.5 Fill Out NA

In [ ]:
#correncia_latitude                 1581
#ocorrencia_longitude                1581
#
#ocorrencia_hora                        1
#investigacao_aeronave_liberada      1952
#investigacao_status                  354
#divulgacao_relatorio_numero          832
#
#divulgacao_dia_publicacao           4740
#
#aeronave_voo_origem                    1
#aeronave_voo_destino                   1
#fator_nome                          5181
#fator_aspecto                       5181
#fator_condicionante                 5181
#fator_area                          5181
#recomendacao_numero                 6246
#recomendacao_dia_assinatura         6246
#recomendacao_dia_encaminhamento     6246
#recomendacao_dia_feedback          10353
#recomendacao_conteudo               6247
#recomendacao_status                 6246
#recomendacao_destinatario_sigla     6246
#recomendacao_destinatario           6246

In [197]:
df2 = df1.copy()

In [218]:
#correncia_latitude  como todas com na tem 'ocorrencia_cidade', colocar 0 para manter a coluna
df2['ocorrencia_latitude'] = df2['ocorrencia_latitude'].cat.add_categories("0").fillna("0")

In [170]:
df1.loc[((df1['ocorrencia_latitude'].isna()) ) , :]


,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,BRASIL,SBBE,03/01/2010,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,COLISÃO COM AVE,COLISÃO COM AVE,BIRD,PRMAB,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A320-232,A320,JATO,BIMOTOR,70000,70000,184,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,REGULAR,LEVE,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,SBRJ,03/01/2010,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,COLISÃO COM AVE,COLISÃO COM AVE,BIRD,PRMBW,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A319-132,A319,JATO,BIMOTOR,75500,75500,154,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,INDETERMINADA,REGULAR,LEVE,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,BRASIL,SBPK,05/01/2010,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,OUTROS,OUTROS,OTHR,PPPSB,REGULAR,AVIÃO,EMBRAER,EMB-120ER,E120,TURBOÉLICE,BIMOTOR,11990,11990,34,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,TÁXI,REGULAR,NENHUM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,39789,39789,39789,39789,39789,INCIDENTE,NaN,NaN,SÃO PEDRO DO SUL,RS,BRASIL,****,10/01/2010,21:30:00,***,NaN,NaN,NÃO,NaN,0,1,NÃO,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,PTGPL,ESPECIALIZADA,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-201,IPAN,PISTÃO,MONOMOTOR,1800,1800,1,0,BRASIL,BRASIL,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,CHEQUE DE MOTOR OU ROTOR,ESPECIALIZADA,LEVE,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,40414,40414,40414,40414,40414,INCIDENTE,NaN,NaN,BRASÍLIA,DF,BRASIL,SBBR,10/01/2010,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,COLISÃO COM AVE,COLISÃO COM AVE,BIRD,PRMAY,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A320-232,A320,JATO,BIMOTOR,77000,77000,166,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,INDETERMINADA,REGULAR,LEVE,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11297,53227,53227,53227,53227,53227,INCIDENTE,NaN,NaN,FLORIANÓPOLIS,SC,BRASIL,SBFL,02/04/2015,01:10:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,PRFLM,INSTRUÇÃO,AVIÃO,PIPER AIRCRAFT,PA-34-200,PA34,PISTÃO,BIMOTOR,1905,1905,6,0,BRASIL,BRASIL,AVIÃO,INSTRUÇÃO,HERCÍLIO LUZ,HERCÍLIO LUZ,TÁXI,INSTRUÇÃO,NENHUM,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11319,53228,53228,53228,53228,53228,

In [219]:
df2['ocorrencia_latitude'].isna().sum()


0

1897.1999999999998

In [46]:
df1.ocorrencia_hora.unique()

array(['12:00:00', '11:05:00', '03:00:00', '17:30:00', '19:25:00',
       '17:53:00', '18:40:00', '12:30:00', '23:15:00', '20:00:00',
       '21:30:00', '14:50:00', '17:00:00', '14:30:00', '21:22:00',
       '15:00:00', '18:23:00', '17:45:00', '12:27:00', '19:32:00',
       '18:30:00', '13:20:00', '12:16:00', '18:58:00', '13:10:00',
       '20:45:00', '13:00:00', '16:36:00', '03:30:00', '16:15:00',
       '16:20:00', '17:05:00', '13:50:00', '16:50:00', '11:25:00',
       '20:06:00', '16:00:00', '10:30:00', '19:47:00', '21:00:00',
       '17:55:00', '12:55:00', '09:30:00', '17:20:00', '21:09:00',
       '22:30:00', '17:40:00', '22:59:00', '11:30:00', '02:09:00',
       '12:33:00', '13:55:00', '23:50:00', '09:03:00', '15:01:00',
       '16:48:00', '16:25:00', '20:07:00', '10:10:00', '10:20:00',
       '13:47:00', '19:42:00', '19:37:00', '11:17:00', '11:19:00',
       '22:32:00', '14:40:00', '09:00:00', '19:35:00', '00:49:00',
       '19:18:00', '06:17:00', '18:10:00', '21:05:00', '09:57: